In [13]:
import pandas as pd
import re
pd.options.display.max_colwidth = 200
df = pd.read_pickle("../data/preprocessed/diagnoses.pkl")
diag_unique_patients = df[["patient_id"]].drop_duplicates()
print(f"N. unique patients in diagnoses: {len(diag_unique_patients)}")
print(f"N. rows diag: {len(df)}")

N. unique patients in diagnoses: 15663
N. rows diag: 138718


In [14]:
df_age_sex = pd.read_excel("../data/raw/hackath 112022 - Age SEX CKD.xlsx")
df_bmi = pd.read_excel("../data/raw/hackath 112022 - BMI weight height CKD 1.xlsx")

In [15]:
# validate NAs and how the dataset match on top of each other
print("### NAs age sex")
print(df_age_sex.isna().sum())
print("### NAs bmi")
print(df_bmi.isna().sum())

age_unique_patients = df_age_sex[["Patient"]].drop_duplicates()
bmi_unique_patients = df_bmi[["Patient"]].drop_duplicates()
print("### Datasets joinability")
print(f'{len(diag_unique_patients.merge(age_unique_patients, how = "inner", left_on = ["patient_id"], right_on = "Patient"))/len(diag_unique_patients)*100} % have age sex info')
print(f'{len(diag_unique_patients.merge(bmi_unique_patients, how = "inner", left_on = ["patient_id"], right_on = "Patient"))/len(diag_unique_patients)*100} % have bmi info')

### NAs age sex
Patient    0
Sex        0
Age        0
dtype: int64
### NAs bmi
Patient             0
date                0
BMI                 0
Weight (kg)         0
Height (cm)    141417
dtype: int64
### Datasets joinability
99.32324586605375 % have age sex info
91.5788801634425 % have bmi info


In [16]:
df_age_sex.rename(columns = {"Patient": "patient_id", "Sex": "sex", "Age": "age_2022"}, inplace = True)
df = df.merge(df_age_sex, on = "patient_id", how = "left")
df["age_date"] = pd.to_datetime(pd.Timestamp.now())
df["age"] = df["age_2022"] -  pd.to_timedelta(df["age_date"] - df["date"]).dt.days/365
df

,patient_id,date,main_code,main_description,other_code,other_description,is_ckd,is_dia,ckd_stadium,min_date,...,is_dia_patient,date_range,min_ckd_date,min_dia_date,is_ckd_cum,is_dia_cum,sex,age_2022,age_date,age
0,209536,2015-01-29,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,NaN,2015-01-29,...,0,2680 days,NaT,NaT,0,0,F,38.9,2022-11-26 10:42:34.826636,31.069863
1,209536,2015-08-06,Z01.4,Gynekologické vyšetření (všeobecné)(rutinní),N83.1,Cysta žlutého tělíska,0,0,NaN,2015-01-29,...,0,2680 days,NaT,NaT,0,0,F,38.9,2022-11-26 10:42:34.826636,31.587671
2,209536,2022-01-10,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,NaN,2015-01-29,...,0,2680 days,NaT,NaT,0,0,F,38.9,2022-11-26 10:42:34.826636,38.023288
3,209536,2022-04-04,D25.1,Intramurální leiomyom dělohy,NaN,NaN,0,0,NaN,2015-01-29,...,0,2680 days,NaT,NaT,0,0,F,38.9,2022-11-26 10:42:34.826636,38.253425
4,209536,2022-06-01,Z01.4,Gynekologické vyšetření (všeobecné)(rutinní),D25.1,Intramurální leiomyom dělohy,0,0,NaN,2015-01-29,...,0,2680 days,NaT,NaT,0,0,F,38.9,2022-11-26 10:42:34.826636,38.412329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138713,1217771,2015-11-11,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",K20,Zánět jícnu - ezofagitida,0,0,NaN,2015-11-11,...,0,2452 days,NaT,NaT,0,0,M,42.9,2022-11-26 10:42:34.826636,35.853425
138714,1217771,2022-07-29,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",I10,Esenciální (primární) hypertenze,0,0,NaN,2015-11-11,...,0,2452 days,NaT,NaT,0,0,M,42.9,2022-11-26 10:42:34.826636,42.571233
138715,1217771,2022-07-29,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",NaN,Hypercholesterolemie,0,0,NaN,2015-11-11,...,0,2452 days,NaT,NaT,0,0,M,42.9,2022-11-26 10:42:34.826636,42.571233
138716,1217771,2022-03-25,K50.8,"Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",NaN,NaN,0,0,NaN,2015-11-11,...,0,2452 days,NaT,NaT,0,0,M,42.9,2022-11-26 10:42:34.826636,42.226027


In [17]:
df_bmi["bmi_fom"] = df_bmi['date'].dt.normalize() - pd.offsets.MonthBegin(1)
df_bmi.rename(columns = {"BMI": "bmi", "Patient": "patient_id"}, inplace = True)
df_bmi

,patient_id,date,bmi,Weight (kg),Height (cm),bmi_fom
0,324729,2015-04-08,29.4,93.0,NaN,2015-04-01
1,324729,2015-08-28,29.4,93.0,NaN,2015-08-01
2,324729,2016-02-03,26.2,83.0,NaN,2016-02-01
3,324729,2016-03-30,30.1,90.0,173.0,2016-03-01
4,324729,2016-05-02,30.1,90.0,173.0,2016-05-01
...,...,...,...,...,...,...
153246,335472,2021-02-22,30.9,98.0,NaN,2021-02-01
153247,335472,2021-09-13,30.9,98.0,NaN,2021-09-01
153248,335472,2022-03-21,30.6,97.0,NaN,2022-03-01
153249,335472,2022-05-13,29.5,93.4,NaN,2022-05-01


In [18]:
date_range = pd.DataFrame({"fom": pd.date_range(start=df["date"].min(), end=df["date"].max(), freq="MS",inclusive="both")})
diag_unique_patients["key"] = 0
date_range['key'] = 0
skeleton = diag_unique_patients.merge(date_range, on = "key", how = "inner")
skeleton_bmi = skeleton.merge(df_bmi, left_on = ["patient_id", "fom"], right_on = ["patient_id", "bmi_fom"], how = "left")
skeleton_bmi = skeleton_bmi.drop_duplicates(subset = ["patient_id", "fom"])
print(skeleton_bmi.isna().sum())
skeleton_bmi["bmi"] = skeleton_bmi.groupby('patient_id')["bmi"].apply(lambda group: group.interpolate(method='linear', limit_area = "inside"))
skeleton_bmi["bmi"] = skeleton_bmi.groupby('patient_id')['bmi'].apply(lambda x: x.bfill().ffill())
print(skeleton_bmi.isna().sum())
#z.merge(df, on = "key")
# TODO: Add fom to df
# bmi to z + interpolation

patient_id           0
key                  0
fom                  0
date           1362215
bmi            1362215
Weight (kg)    1362215
Height (cm)    1503588
bmi_fom        1362215
dtype: int64


C:\Users\pavel\AppData\Local\Temp\ipykernel_3280\3218017658.py:7: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  skeleton_bmi["bmi"] = skeleton_bmi.groupby('patient_id')["bmi"].apply(lambda group: group.interpolate(method='linear', limit_area = "inside"))
C:\Users\pavel\AppData\Local\Temp\ipykernel_3280\3218017658.py:8: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior

patient_id           0
key                  0
fom                  0
date           1362215
bmi             126624
Weight (kg)    1362215
Height (cm)    1503588
bmi_fom        1362215
dtype: int64


In [20]:
skeleton_bmi.drop(columns = ["date"], inplace = True)

In [28]:
skeleton_bmi.drop_duplicates(subset = ["patient_id", "fom"]).shape

(1503648, 7)

In [21]:
df["date_fom"] = df['date'].dt.normalize() - pd.offsets.MonthBegin(1)
df = df.merge(skeleton_bmi, left_on = ["patient_id", "date_fom"], right_on = ["patient_id", "bmi_fom"])

In [26]:
df.drop(columns = ["key", "Weight (kg)", "Height (cm)", "age_date", "bmi_fom"], inplace = True, errors="ignore")
df

,patient_id,date,main_code,main_description,other_code,other_description,is_ckd,is_dia,ckd_stadium,min_date,...,min_ckd_date,min_dia_date,is_ckd_cum,is_dia_cum,sex,age_2022,age,date_fom,fom,bmi
0,209536,2015-01-29,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,NaN,2015-01-29,...,NaT,NaT,0,0,F,38.9,31.069863,2015-01-01,2015-01-01,45.1
1,209536,2022-01-10,E06.3,Autoimunitní tyroiditida,E78.0,Čistá hypercholesterolemie,0,0,NaN,2015-01-29,...,NaT,NaT,0,0,F,38.9,38.023288,2022-01-01,2022-01-01,31.0
2,8451,2015-01-08,E10.9,"Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",NaN,Autoimunní thyreoiditis,0,1,NaN,2015-01-08,...,NaT,2015-01-08,0,0,M,62.8,54.912329,2015-01-01,2015-01-01,26.9
3,8451,2015-01-08,E10.9,"Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",NaN,Perniciózní anemie,0,1,NaN,2015-01-08,...,NaT,2015-01-08,0,0,M,62.8,54.912329,2015-01-01,2015-01-01,26.9
4,8451,2015-01-08,E10.9,"Diabetes mellitus 1. typu, léč. intenzifikovaným inzulínovým režimem",NaN,"Surditas vlevo, tinnitus vlevo",0,1,NaN,2015-01-08,...,NaT,2015-01-08,0,0,M,62.8,54.912329,2015-01-01,2015-01-01,26.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95612,1245853,2022-07-14,K51.0,Ulcerózní pankolitida (5/2015),NaN,Stav po borelióze (6/2022),0,0,NaN,2015-06-01,...,NaT,NaT,0,0,M,30.9,30.530137,2022-07-01,2022-07-01,21.2
95613,1217771,2015-11-11,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",K20,Zánět jícnu - ezofagitida,0,0,NaN,2015-11-11,...,NaT,NaT,0,0,M,42.9,35.853425,2015-11-01,2015-11-01,25.3
95614,1217771,2022-07-29,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",I10,Esenciální (primární) hypertenze,0,0,NaN,2015-11-11,...,NaT,NaT,0,0,M,42.9,42.571233,2022-07-01,2022-07-01,26.9
95615,1217771,2022-07-29,K50.8,"Jiná Crohnova nemoc - postižení ileocéka a sigmatu, st.p. chir. řešení perianál. píštěle",NaN,Hypercholesterolemie,0,0,NaN,2015-11-11,...,NaT,NaT,0,0,M,42.9,42.571233,2022-07-01,2022-07-01,26.9


In [124]:
pd.options.display.max_rows = 50
skeleton_bmi.loc[skeleton_bmi["patient_id"] == 1217771, :]

,patient_id,key,fom,date,bmi,Weight (kg),Height (cm),bmi_fom
1515308,1217771,0,2015-01-01,NaT,NaN,NaN,NaN,NaT
1515309,1217771,0,2015-02-01,NaT,NaN,NaN,NaN,NaT
1515310,1217771,0,2015-03-01,NaT,NaN,NaN,NaN,NaT
1515311,1217771,0,2015-04-01,NaT,NaN,NaN,NaN,NaT
1515312,1217771,0,2015-05-01,NaT,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...
1515407,1217771,0,2022-08-01,2022-08-31,27.8,91.0,NaN,2022-08-01
1515408,1217771,0,2022-09-01,NaT,NaN,NaN,NaN,NaT
1515409,1217771,0,2022-10-01,NaT,NaN,NaN,NaN,NaT
1515410,1217771,0,2022-11-01,NaT,NaN,NaN,NaN,NaT
